## RNN을 이용하여 텍스트 생성하기

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# 패딩을 위한 0을 고려하여 +1
vocab_size = len(tokenizer.word_index) + 1
print(f'단어 집합의 크기 : {vocab_size}')
print(tokenizer.word_index)

단어 집합의 크기 : 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [7]:
print(f'단어 집합의 크기 : {vocab_size}')

단어 집합의 크기 : 12


[texts_to_sequences 공식 문서](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences)
* Transforms each text in texts to a sequence of integers.

In [8]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

print(f'학습에 사용할 샘플의 개수: {len(sequences)}')

학습에 사용할 샘플의 개수: 11


### 위 반복문 해부

In [14]:
sequences = list()
for line in text.split('\n'): # 줄바꿈 문자를 기준으로 문장 토큰화
    # 단어장의 index로 문장 인코딩 
    # 경마장에 있는 말이 뛰고 있다 = [[2, 3, 1, 4, 5]]
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

## 다시 돌아와서

In [16]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [17]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print(f'샘플의 최대 길이 : {max_len}')

샘플의 최대 길이 : 6


In [18]:
# pad_sequences : 0을 활용하여 모든 샘플에 대해 길이를 맞춤
# 아래 코드는 모든 시퀀스의 길이를 6으로 맞추기 위해 0을 샘플의 앞에 채움
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [27]:
# 각 샘플의 마지막 단어를 label로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]
print(X)
print()
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]

[ 3  1  4  5  1  7  1  9 10  1 11]


In [28]:
# RNN 모델에 훈련 데이터로 사용하기 위해 One-hot encoding
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

## Modeling

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [30]:
# 하이퍼파라미터 : embedding_dim = 10, hidden_units = 32
embedding_dim = 10
hidden_units = 32

# 다대일 RNN
# 마지막 단어를 예측하는 다중 클래스 분류 모델
# 활성화 함수 : 소프트맥스 함수
# 손실 함수 : 크로스 엔트로피 함수
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-22 01:52:26.980608: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-22 01:52:26.980859: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


2023-02-22 01:52:27.667869: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-22 01:52:28.048621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 - 2s - loss: 2.4886 - accuracy: 0.1818 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.4750 - accuracy: 0.2727 - 41ms/epoch - 41ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4615 - accuracy: 0.2727 - 40ms/epoch - 40ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4481 - accuracy: 0.2727 - 39ms/epoch - 39ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4345 - accuracy: 0.2727 - 70ms/epoch - 70ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4208 - accuracy: 0.2727 - 39ms/epoch - 39ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4068 - accuracy: 0.3636 - 45ms/epoch - 45ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3924 - accuracy: 0.3636 - 53ms/epoch - 53ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3775 - accuracy: 0.4545 - 42ms/epoch - 42ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3621 - accuracy: 0.4545 - 36ms/epoch - 36ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3461 - accuracy: 0.4545 - 42ms/epoch - 42ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3293 - accuracy: 0.4545 - 40ms/epoch - 40ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3117 - accuracy: 0.45

## 문장 생성 함수

In [31]:
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')

        # encoding 값을 이용하여 다음 단어 y를 예측 
        result = model.predict(encoded, verbose=0)

        # 확률이 가장 높은 단어의 index
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items():
            # 인덱스가 동일한 단어가 있다면 word 할당, break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어
        current_word = current_word + ' ' + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word
    
    sentence = init_word + sentence
    return sentence

In [32]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

2023-02-22 02:04:44.691586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


경마장에 있는 말이 뛰고 있다


In [33]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의 말이 법이다


In [34]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 곱다


## LSTM을 이용한 텍스트 생성

In [35]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [38]:
df = pd.read_csv('../../data/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [87]:
print('열의 개수: ',len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [88]:
# 열의 제목에 Null 값이 있는지 확인
print(df['headline'].isnull().values.any())

False


In [89]:
# 헤드라인의 값들을 리스트로 저장
headline = df['headline'].values.tolist()
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [90]:
print(f'총 샘플의 개수 : {len(headline)}')

총 샘플의 개수 : 1324


In [91]:
# "Unknown" 데이터 제거
headline = [word for word in headline if word != "Unknown"]
print(f'노이즈값 제거 후 샘플의 개수 : {len(headline)}')
headline[:5]

노이즈값 제거 후 샘플의 개수 : 1214


['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [92]:
def repreprocessing(raw_sentence):
    '''
    구두점 제거, 소문자화를 진행하는 전처리 함수
    '''
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [93]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)

vocab_size = len(tokenizer.word_index) + 1
print(f'단어 집합의 크기 : {vocab_size}')

단어 집합의 크기 : 3494


In [94]:
sequences = list()

for sentence in preprocessed_headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [95]:
max_len = max(len(l) for l in sequences)
print(f'샘플의 최대 길이 : {max_len}')
print('샘플의 최대 길이 :', len(max(sequences, key=len)))

샘플의 최대 길이 : 24
샘플의 최대 길이 : 24


In [96]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [97]:
# 가장 우측 단어 label로 분리
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(X[:3])
print(y[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


In [98]:
# label y one-hot encoding
y = to_categorical(y, num_classes=vocab_size)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## 모델링

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [100]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200


2023-02-22 02:25:20.044260: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 02:25:20.209649: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 02:25:21.310343: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


244/244 - 8s - loss: 7.6596 - accuracy: 0.0246 - 8s/epoch - 32ms/step
Epoch 2/200
244/244 - 4s - loss: 7.1113 - accuracy: 0.0291 - 4s/epoch - 18ms/step
Epoch 3/200
244/244 - 4s - loss: 6.9803 - accuracy: 0.0338 - 4s/epoch - 18ms/step
Epoch 4/200
244/244 - 4s - loss: 6.8678 - accuracy: 0.0388 - 4s/epoch - 18ms/step
Epoch 5/200
244/244 - 4s - loss: 6.7345 - accuracy: 0.0432 - 4s/epoch - 18ms/step
Epoch 6/200
244/244 - 4s - loss: 6.5714 - accuracy: 0.0452 - 4s/epoch - 17ms/step
Epoch 7/200
244/244 - 4s - loss: 6.3905 - accuracy: 0.0525 - 4s/epoch - 18ms/step
Epoch 8/200
244/244 - 4s - loss: 6.1938 - accuracy: 0.0566 - 4s/epoch - 17ms/step
Epoch 9/200
244/244 - 4s - loss: 5.9938 - accuracy: 0.0591 - 4s/epoch - 18ms/step
Epoch 10/200
244/244 - 4s - loss: 5.8010 - accuracy: 0.0666 - 4s/epoch - 18ms/step
Epoch 11/200
244/244 - 4s - loss: 5.6194 - accuracy: 0.0704 - 4s/epoch - 18ms/step
Epoch 12/200
244/244 - 5s - loss: 5.4489 - accuracy: 0.0763 - 5s/epoch - 19ms/step
Epoch 13/200
244/244 - 5s

In [101]:
# 문장 생성 함수
def sentence_generation(model, tokenizer, current_word, n):
    init_word = current_word
    sentence = ''

    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
        
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            if index == result:
                break

        current_word = current_word + ' '  + word
        sentence = sentence + ' ' + word
        
    sentence = init_word + sentence
    return sentence

In [102]:
print(sentence_generation(model, tokenizer, 'i', 10))

2023-02-22 02:40:59.107966: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-22 02:40:59.168904: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


i want to be rich and im not sorry draws risks


In [103]:
print(sentence_generation(model, tokenizer, 'how', 10))

how can a doctor grapple with the epidemic of cost coverage
